# Universal Code Forge: Multi-Language Converter & Generator
A multi-modal system using OpenRouter, Gradio, and performance-optimized code generation logic.

In [7]:
import gradio as gr
import os
from openai import OpenAI

client = OpenAI(base_url="https://openrouter.ai/api/v1", api_key=os.getenv('OPENROUTER_API_KEY'))

In [15]:
MODELS = [
    "qwen/qwen-2.5-coder-32b",
    "deepseek/deepseek-coder-v2",
    "anthropic/claude-3.5-sonnet",
    "openai/gpt-4o",
    "bigcode/starcoder2-15b"
]

LANGUAGE_CONFIG = {
    "Rust": {"ext": "rs"},
    "C++": {"ext": "cpp"},
    "JavaScript": {"ext": "js"},
    "PHP": {"ext": "php"},
    "Go": {"ext": "go"},
    "Zig": {"ext": "zig"},
    "Java": {"ext": "java"},
    "TypeScript": {"ext": "ts"},
    "Python": {"ext": "py"},
    "C#": {"ext": "cs"},
    "Swift": {"ext": "swift"},
    "Kotlin": {"ext": "kt"},
    "Ruby": {"ext": "rb"},
    "Elixir": {"ext": "ex"}
}

In [9]:
def get_system_prompt(language, docs):
    doc_instr = "Include thorough doc comments." if docs else "Do not include any doc comments or excessive explanation."
    return f"""
    You are an expert polyglot engineer. Generate {language} code optimized for extreme runtime performance.
    - {doc_instr}
    - Respond only with the raw code.
    """

In [10]:
def process_request(fromLanguage, toLanguage, include_docs, model, code_in, prompt_in):
    system_prompt = get_system_prompt(toLanguage, include_docs)
    
    # Determine if converting (code_in) or generating (prompt_in)
    task = f"Convert this {fromLanguage} code: {code_in}" if code_in else f"Generate {toLanguage} code for: {prompt_in}"
    
    response = client.chat.completions.create(
        model=model,
        messages=[{"role": "system", "content": system_prompt}, {"role": "user", "content": task}],
        temperature=0.1
    )
    return response.choices[0].message.content

In [ ]:
with gr.Blocks(title="Universal Code Forge") as demo:
    gr.Markdown("# 🛠️ Universal Code Forge")
    
    with gr.Row():
        f_lang = gr.Dropdown(list(LANGUAGE_CONFIG.keys()), label="Source Language")
        t_lang = gr.Dropdown(list(LANGUAGE_CONFIG.keys()), label="Target Language")
        doc_toggle = gr.Checkbox(label="Include Doc Comments", value=False)
        model_sel = gr.Dropdown(MODELS, label="Model")
    
    with gr.Tabs():
        with gr.TabItem("Convert Code"): 
            code_in = gr.Code(label="Input Code")
        with gr.TabItem("Generate from Prompt"): 
            prompt_in = gr.Textbox(label="Request")
    
    out_code = gr.Code(label="Optimized Output")
    btn = gr.Button("Forge Code")
    
    btn.click(process_request, inputs=[f_lang, t_lang, doc_toggle, model_sel, code_in, prompt_in], outputs=out_code)

demo.launch(inbrowser=True)